# Micromobility in Minneapolis, MN: Spatiotemporal Prediction of Origins & Destinations

## Notebook 1: ETL and Data Wrangling

### Luke Zaruba, University of Minnesota - MGIS Program

##### March 31, 2023

*Micromobility has been a recent interest of transportation planners and urban residents alike. The most popular implementations of micromobility in Minneapolis has been through motorized scooters from companies like Lime (Uber), Bird, or Lyft. There are comprehensive datasets available containing trip information for all scooters within the City, but using the data for understanding travel patterns and user behaviors has largely remained untouched. The data could have immense value to planners, residents, and other stakeholders, to inform future decision-making and more adequately prepare for how the urban landscape will change to accommodate greater volumes of scooters and other forms of micromobility. My solution is to use the powerful techniques of spatial data science to uncover these patterns and estimate when and where users are traveling **to** and **from**.*

In [1]:
# Ignore Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import Libraries
import arcpy
import os

## Loading Data to File Geodatabase

In [3]:
# Set Paths
working_gdb = r"C:\gitFiles\MicromobilityForecasting\MicromobilityForecastingAPRX\MicromobilityForecastingAPRX.gdb"
data_path = r"C:\gitFiles\MicromobilityForecasting\data\raw"

# Set as Working Envirnoment
arcpy.env.workspace = working_gdb

# Copy Streets to GDB
arcpy.conversion.ExportFeatures(
    in_features = os.path.join(data_path, r"PW_Street_Centerline\PW_Street_Centerline.shp"),
    out_features = os.path.join(working_gdb, "streets_line")
)

# Copy Trips to GDB
arcpy.management.CopyRows(
    in_rows = os.path.join(data_path, "Motorized_Foot_Scooter_Trips_2021.csv"),
    out_table = os.path.join(working_gdb, "trips_table")
)

## Cleaning the Street Centerlines

In [4]:
# Create List of Fields
street_fields = [i.name for i in arcpy.ListFields("streets_line")]

# Create List of Fields to Drop
drop_street_fields = [i for i in street_fields if i not in ("GBSID", "OBJECTID_1", "Shape_Length", "Shape")]

# Drop Unnecessary Fields
arcpy.management.DeleteField(
    in_table = os.path.join(working_gdb, "streets_line"),
    drop_field = drop_street_fields,
    method = "DELETE_FIELDS"
)

In [5]:
# Calculate Centroid Coordinates
arcpy.management.CalculateGeometryAttributes(
    in_features = os.path.join(working_gdb, "streets_line"),
    geometry_property = "X CENTROID_X;Y CENTROID_Y",
    coordinate_system = arcpy.SpatialReference(4236),
    coordinate_format = "DD"
)

In [6]:
# Create Centroid Point Feature Class
arcpy.management.XYTableToPoint(
    in_table = os.path.join(working_gdb, "streets_line"),
    out_feature_class = os.path.join(working_gdb, "street_centroids"),
    x_field = "X",
    y_field = "Y",
    coordinate_system = arcpy.SpatialReference(4326)
)

## Cleaning the Scooter Trips

In [7]:
# Alter Start Time Field
arcpy.management.ConvertTimeField(
    in_table = "trips_table",
    input_time_field = "StartTime",
    input_time_format = "yyyy/MM/dd HH:mm:ss+00",
    output_time_field = "StartTime_Converted",
    output_time_type = "DATE"
)

In [8]:
# Alter End Time Field
arcpy.management.ConvertTimeField(
    in_table = "trips_table",
    input_time_field = "EndTime",
    input_time_format = "yyyy/MM/dd HH:mm:ss+00",
    output_time_field = "EndTime_Converted",
    output_time_type = "DATE"
)

In [9]:
# Restrict to Trips that Start & End on Streets
arcpy.conversion.ExportTable(
    in_table = os.path.join(working_gdb, "trips_table"),
    out_table = os.path.join(working_gdb, "trips_table_cleaned"),
    where_clause = "StartCenterlineType = 'street' AND EndCenterlineType = 'street'"
)

## Joining Scooter Trips to Streets

In [10]:
# Join by Origin ID
arcpy.gapro.JoinFeatures(
    target_layer = os.path.join(working_gdb, "trips_table"),
    join_layer = os.path.join(working_gdb, "street_centroids"),
    output = os.path.join(working_gdb, "origins_table"),
    join_operation = "JOIN_ONE_TO_MANY",
    attribute_relationship="StartCenterlineID GBSID"
)

In [11]:
# Join by Destination ID
arcpy.gapro.JoinFeatures(
    target_layer = os.path.join(working_gdb, "trips_table"),
    join_layer = os.path.join(working_gdb, "street_centroids"),
    output = os.path.join(working_gdb, "destinations_table"),
    join_operation = "JOIN_ONE_TO_MANY",
    attribute_relationship="EndCenterlineID GBSID"
)

## Convert Scooter Trips to Feature Classes

In [12]:
# Convert Origins to Feature Class
arcpy.management.XYTableToPoint(
    in_table = os.path.join(working_gdb, "origins_table"),
    out_feature_class = os.path.join(working_gdb, "origins_points"),
    x_field = "X",
    y_field = "Y",
    coordinate_system = arcpy.SpatialReference(4326)
)

# Convert Destinations to Feature Class
arcpy.management.XYTableToPoint(
    in_table = os.path.join(working_gdb, "destinations_table"),
    out_feature_class = os.path.join(working_gdb, "destinations_points"),
    x_field = "X",
    y_field = "Y",
    coordinate_system = arcpy.SpatialReference(4326)
)

## Creating Space Time Cubes

In [13]:
# Project to PCS for STC to Work Properly
arcpy.management.Project(
    in_dataset="origins_points",
    out_dataset= os.path.join(working_gdb, "origins_points_utm15"),
    out_coor_system= arcpy.SpatialReference(26915),
    transform_method="WGS_1984_(ITRF00)_To_NAD_1983",
    in_coor_system = arcpy.SpatialReference(4326)
)

arcpy.management.Project(
    in_dataset="destinations_points",
    out_dataset= os.path.join(working_gdb, "destinations_points_utm15"),
    out_coor_system= arcpy.SpatialReference(26915),
    transform_method="WGS_1984_(ITRF00)_To_NAD_1983",
    in_coor_system = arcpy.SpatialReference(4326)
)

In [14]:
# Create Origin STC
arcpy.stpm.CreateSpaceTimeCube(
    in_features = "origins_points_utm15",
    output_cube = r"C:\gitFiles\MicromobilityForecasting\data\outputs\origins_stc.nc",
    time_field = "StartTime_Converted",
    time_step_interval = "30 Minutes",
    time_step_alignment = "END_TIME",
    aggregation_shape_type="FISHNET_GRID"
)

In [15]:
# Create Destination STC
arcpy.stpm.CreateSpaceTimeCube(
    in_features = "destinations_points_utm15",
    output_cube = r"C:\gitFiles\MicromobilityForecasting\data\outputs\destinations_stc.nc",
    time_field = "EndTime_Converted",
    time_step_interval = "30 Minutes",
    time_step_alignment = "END_TIME",
    aggregation_shape_type="FISHNET_GRID"
)